# Projecct 2 Part 3A - EDA

*Christina Brockway*

#### Business Issues

-  Use TMDB database
-  Extract budget, revenue, and MPAA Rating (Certification) data
-  Perform test extraction on movies from 2001 and 2002
-  Combine final API data into 1 dataframe
-  Create visualiztions to answer questions

### Imports

In [3]:
#Import packages
import os, time, json
import tmdbsimple as tmdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm_notebook

#Set configs
pd.set_option('display.max_column', None)

### Define custom functions


In [4]:
# Explore Categorical Data

def explore_categorical(df, x, fillna = True, placeholder = 'MISSING',
                        figsize = (6,4), order = None):
  """Source: https://login.codingdojo.com/m/606/13765/117604"""
  # Make a copy of the dataframe and fillna
  temp_df = df.copy()
  # Before filling nulls, save null value counts and percent for printing
  null_count = temp_df[x].isna().sum()
  null_perc = null_count/len(temp_df)* 100
  # fillna with placeholder
  if fillna == True:
    temp_df[x] = temp_df[x].fillna(placeholder)
  # Create figure with desired figsize
  fig, ax = plt.subplots(figsize=figsize)
  # Plotting a count plot
  sns.countplot(data=temp_df, x=x, ax=ax, order=order)
  #Labels for barplot values
  for container in ax.containers:
      ax.bar_label(container)
  # Rotate Tick Labels for long names
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
  # Add a title with the feature name included
  ax.set_title(f"Column: {x}")

  # Fix layout and show plot (before print statements)
  fig.tight_layout()
  plt.show()

  # Print null value info
  print(f"- NaN's Found: {null_count} ({round(null_perc,2)}%)")
  # Print cardinality info
  nunique = temp_df[x].nunique()
  print(f"- Unique Values: {nunique}")
  # First find value counts of feature
  val_counts = temp_df[x].value_counts(dropna=False)
  # Define the most common value
  most_common_val = val_counts.index[0]
  # Define the frequency of the most common value
  freq = val_counts.values[0]
  # Calculate the percentage of the most common value
  perc_most_common = freq / len(temp_df) * 100
  # Print the results
  print(f"- Most common value: '{most_common_val}' occurs {freq} times ({round(perc_most_common,2)}%)")
  # print message if quasi-constant or constant (most common val more than 98% of data)
  if perc_most_common > 98:
    print(f"\n- [!] Warning: '{x}' is a constant or quasi-constant feature and should be dropped.")
  else:
    print("- Not constant or quasi-constant.")
  return fig, ax



*Source for barplot labels:  https://stackoverflow.com/questions/55104819/display-count-on-top-of-seaborn-barplot*

In [5]:
# Updated plot_categorical_vs_target function that includes option for classification task
def plot_categorical_vs_target(df, x, y, figsize=(6,4),
                            fillna = True, placeholder = 'MISSING',
                            order = None, target_type='reg'):
  # Make a copy of the dataframe and fillna
  temp_df = df.copy()
  # fillna with placeholder
  if fillna == True:
    temp_df[x] = temp_df[x].fillna(placeholder)
  # or drop nulls prevent unwanted 'nan' group in stripplot
  else:
    temp_df = temp_df.dropna(subset=[x])
  # Create the figure and subplots
  fig, ax = plt.subplots(figsize=figsize)
  # REGRESSION-TARGET PLOT
  if target_type=='reg': #Added if statement here
    # Barplot
    sns.barplot(data=temp_df, x=x, y=y, ax=ax, order=order, alpha=0.6,
                linewidth=1, edgecolor='black', errorbar=None)
    # Boxplot
    sns.stripplot(data=temp_df, x=x, y=y, hue=x, ax=ax,
                  order=order, hue_order=order, legend=False,
                  edgecolor='white', linewidth=0.5,
                  size=3,zorder=0)
  # CLASSIFICATION-TARGET PLOT # This is the new code for the classification task
  elif target_type=='class':
    ax = sns.histplot(data=temp_df, hue=y, x=x, stat='percent',  multiple='fill')
  # Rotate xlabels
  ax.set_xticks(ax.get_xticks()) # Added this to prevent a bug
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
  # Add a title
  ax.set_title(f"{x} vs. {y}")
  fig.tight_layout()
  return fig, ax


### Load data

In [7]:
df2001=pd.read_csv("MovieData/tmdb_api_results 2001.csv", sep='\t', low_memory=False)
df2001.head(2)

,"[{""imdb_id"": 0}","{""adult"": false","""backdrop_path"": ""/tJLV3BAlHOgscVOrA99Wnb2gAef.jpg""","""belongs_to_collection"": null","""budget"": 48000000","""genres"": [{""id"": 10749","""name"": ""Romance""}","{""id"": 14","""name"": ""Fantasy""}","{""id"": 35","""name"": ""Comedy""}]","""homepage"": """"","""id"": 11232","""imdb_id"": ""tt0035423""","""original_language"": ""en""","""original_title"": ""Kate & Leopold""","""overview"": ""When her scientist ex-boyfriend discovers a portal to travel through time -- and brings back a 19th-century nobleman named Leopold to prove it -- a skeptical Kate reluctantly takes responsibility for showing Leopold the 21st century. The more time Kate spends with Leopold",the harder she falls for him. But if he doesn't return to his own time,"his absence will forever alter history.""","""popularity"": 12.872","""poster_path"": ""/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg""","""production_companies"": [{""id"": 85","""logo_path"": null","""name"": ""Konrad Pictures""","""origin_country"": """"}","{""id"": 14.1","""logo_path"": ""/m6AHu84oZQxvq7n1rsvMNJIAsMu.png""","""name"": ""Miramax""","""origin_country"": ""US""}]","""production_countries"": [{""iso_3166_1"": ""US""","""name"": ""United States of America""}]","""release_date"": ""2001-12-25""","""revenue"": 76019048","""runtime"": 118","""spoken_languages"": [{""english_name"": ""French""","""iso_639_1"": ""fr""","""name"": ""Fran\u00e7ais""}","{""english_name"": ""English""","""iso_639_1"": ""en""","""name"": ""English""}","{""english_name"": ""Italian""","""iso_639_1"": ""it""","""name"": ""Italiano""}]","""status"": ""Released""","""tagline"": ""If they lived in the same century","they'd be perfect for each other.""","""title"": ""Kate & Leopold""","""video"": false","""vote_average"": 6.32","""vote_count"": 1237","""certification"": ""PG-13""}","{""adult"": false.1","""backdrop_path"": null","""belongs_to_collection"": null.1","""budget"": 0","""genres"": [{""id"": 53","""name"": ""Thriller""}","{""id"": 28","""name"": ""Action""}]","""homepage"": """".1","""id"": 151007","""imdb_id"": ""tt0114447""","""original_language"": ""en"".1","""original_title"": ""The Silent Force""","""overview"": ""An elite Federal force whose objective is to eliminate organized crime is violently assassinated by an evil Asian crime lord. The sole survivor of the group teams up with his late partner's widow to strike back at the crime lord and his son.""","""popularity"": 3.472","""poster_path"": ""/u0njmfXD3dXxQfeykBHBuFQOYqD.jpg""","""production_companies"": []","""production_countries"": []","""release_date"": ""2001-01-01""","""revenue"": 0","""runtime"": 90","""spoken_languages"": [{""english_name"": ""English""","""iso_639_1"": ""en"".1","""name"": ""English""}]","""status"": ""Released"".1","""tagline"": ""They left him for dead... They should have finished the job""","""title"": ""The Silent Force""","""video"": false.1","""vote_average"": 5.0","""vote_count"": 3}","{""adult"": false.2","""backdrop_path"": ""/rFpHBidSlhjflmnLu7BZilyKeQR.jpg""","""belongs_to_collection"": null.2","""budget"": 0.1","""genres"": [{""id"": 28","""name"": ""Action""}","{""id"": 12","""name"": ""Adventure""}","{""id"": 14.2","""name"": ""Fantasy""}.1","{""id"": 878","""name"": ""Science Fiction""}]","""homepage"": """".2","""id"": 73549","""imdb_id"": ""tt0116916""","""original_language"": ""en"".2","""original_title"": ""The Dark Mist""","""overview"": ""With the future of the world hanging in the balance","one fearless hero's quest to solve \""The Riddle of the Chosen\"". As the Lord Protector makes his way through a land of magic and mystery",his allies grow stronger and his enemies more fierce. Will the Lord Protector make it to the center of evil and banish the dark forces that plot destruction,"or will the power of evil overwhelm this mortal man and envelope the entire planet in a shroud of darkness and despair?""","""popularity"": 1.4","""poster_path"": ""/z18E0lukzOtAALmfI

In [8]:
df2002=pd.read_csv("MovieData/tmdb_api_results 2002.csv.gz", sep='\t', low_memory=False)
")
df2002.head(2)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0096056,0.0,/95U3MUDXu4xSCmVLtWgargRipDi.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,109809.0,en,Crime and Punishment,A modern day adaptation of Dostoyevsky's class...,5.05,/2ckMQwDi11TofiNoaE3sHrYbaCh.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2002-06-01,0.0,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Crime and Punishment,0.0,5.3,12.0,NaN


In [9]:
df_to_combine = [df2001, df2001]
df = pd.concat(df_to_combine)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 56907 entries, [{"imdb_id": 0} to  "vote_count": 0}]
dtypes: object(56907)
memory usage: 0.0+ bytes


### Inspect the Data

In [ ]:
df.head(5), df.tail(5)

In [ ]:
#Display the number of rows and columns
df.shape

In [ ]:
# Desplay statistics
df.describe()


In [ ]:
#Display non-null values and dtypes
df.info()

## Exploratory Data Analysis

### Questions:

**How many movies had at least some valid financial information?**
-  values >0 for budget and/or >0 for revenue
-  

In [ ]:
df_budget = df.loc([df['budget'}>0) | (df["revenue"])
df_budget

**How many movies are there in each of the certification categories?**

In [ ]:
explore_categorical(df_budget['certification']);

*Source:  https://stackoverflow.com/questions/30482071/how-to-calculate-mean-values-grouped-on-another-column*

**What is the average revenue per certification category?**

In [ ]:
df.groupby('certification', as_index=False)['revenue'].mean()

x=df_budget['certification']
y=df_budget['revenue']
plot_categorical_vs_target(df_budget, x,y);

**What is the average budget per certification category?**

In [ ]:
df.groupby('certification', as_index=False)['budget'].mean()

In [ ]:
x=df_budget['certification']
y=df_budget['budget']
plot_categorical_vs_target(df_budget, x,y);

In [ ]:
# Save dataframe
df_budget.pd.to_csv("/MovieData/tmdb_results_combined.csv.gz", compression = 'gzip', index=False)